In [188]:
from typing import List, Callable # potrzebne do utworzenia genomu, FitnessFunc (https://docs.python.org/3/library/typing.html#typing.Callable)
from random import choices, randint, randrange, random # potrzebne do lodowań
from collections import namedtuple # potrzebne do utworzenia listy przedmiotów

In [129]:
Genome = List[int] # Genom - lista przedmiotów określona binarnie
Population = List[Genome] # Populacja - lista list przedmiotów (genomów)
FitnessFunc = Callable[[Genome], int] # Funkcja dopasowania - składnia Callable() musi być zawsze używana z dokładnie dwiema wartościami: listą argumentów i typem zwracanym.
Thing = namedtuple('Thing', ['name', 'value', 'weight']) # namedtuple() jest specjalnym Pythonowym obiektem, podobnym do słownika

things = [
    Thing('Laptop', 500, 2200),
    Thing('Headphones', 150, 160),
    Thing('Coffe Mug', 60, 350),
    Thing('Notepad', 40, 333),
    Thing('Water Bottle', 30, 192),
]

more_things = [
    Thing('Mints', 5, 25),
    Thing('Socks', 10, 38),
    Thing('Tissues', 15, 80),
    Thing('Phone', 500, 200),
    Thing('Baseball Cap', 100, 70),
] + things

def generate_genome(length: int) -> Genome:
    # "->" oznacza adnotacje do funkcji, mówi ona "third-party" programom oraz nam, jaki typ zwraca dana funkcja
    # jest to nieobligatoryjny zapis
    # https://www.youtube.com/watch?v=k56rEoRjK4k
    # https://peps.python.org/pep-3107/  
    # https://renenyffenegger.ch/notes/development/languages/Python/dunders/__annotations__/index
    return choices([0,1], k = length)

def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)] # podkreślnik "_" jest "poprawniejszym" zapisem w języku iteracji (miast pisania i,j,k,l - jak w matematyce)

def fitness(genome: Genome, things: [Thing], weight_limit: int) -> int:
    # odwołania typu "genome: Genome" służą do wykorzystania wczesniej utworzonych obiektów
    if len(Genome) != len(things):
        raise ValueError("genome and things must be of the same length")
        # jest to niezbędne, w przeciwnym wypadku tablica przedmiotów i tablica genomów [1,0,0,0,1 ...] itp. nie będą się pokrywać
        
    weight = 0
    value = 0
    
    for i, thing in enumerate(things): # enumerate() - ułatwia "przechodzenie" po przedmiotach z listy https://realpython.com/python-enumerate/
        if genome[i] == 1:
            weigth += thing.weight
            value += thing.value
            # sepcjalna własność (odwoływań) funkcji enumerate()
            
            if weigth > weight_limit: # jeżeli przekroczymy limit wagowy, to rozwiązanie "do wyrzucenia"
                return 0
            
    return value

def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    # Dlaczego nie odwołujemy się do naszej funkcji fitness?
    # Jest to związane z architekturą oprogramowania oraz rozdzieleniem problemów.
    # Wcześniej napisana funkcja jest dedykowana problemowi pakowania plecaka (dodatkowe dwa argumenty - rzeczy, dopuszczalna waga)
    return choices(
        population = Population,
        weights = [fitness_func(genome) for genome in population],
        # przez wykorzystanie wag prawdopodobieństwa, podpowiadamy funkcji, aby genomy o większej wadzę wybierał z większym prawdopodobieństwem
        k = 2
    )

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("genome and things must be of the same length")
        
    # jeżeli obie listy genomów będą długości 1, to nic sobie nie pomieszamy
    length = len(a)
    if length < 2:
        return a, b
    
    # mieszanie dwóch genomów w losowym ich "przecięciu"
    p = randint.(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutataion(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome: # z pewnym prawdopodobieństwem będziemy zmieniać 0 -> 1 oraz 1 -> 0
    for _ in range(num):
        index = randrange(len(genome)) # mniej kosztowna funkcja losująca indeks z określonego zakresu
        # https://stackoverflow.com/questions/3540431/what-is-the-difference-between-random-randint-and-randrange
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
        # inna konstrukcja warunku "if" tj. w przypadku wylosowania liczby większej od probability, przypisuje genome[index] = genome[index]
        # w przeciwnym wypadku abs(genome[index] - 1) - nalezy rozważyć oba możliwe przypadki:
        # jeżeli genome[index] = 1, to abs(genome[index] - 1) = abs(1 - 1) = 0
        # jeżeli genome[index] = 0, to abs(genome[index] - 1) = abs(0 - 1) = abs(-1) = 1
        # sprytna zamiana wartości
    return genome

In [130]:
print(generate_genome(7), "\n", generate_population(10, 7), "\n", things, "\n", more_things)

[1, 0, 0, 1, 1, 1, 0] 
 [[1, 0, 0, 1, 1, 1, 0], [1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 1, 1]] 
 [Thing(name='Laptop', value=500, weight=2200), Thing(name='Headphones', value=150, weight=160), Thing(name='Coffe Mug', value=60, weight=350), Thing(name='Notepad', value=40, weight=333), Thing(name='Water Bottle', value=30, weight=192)] 
 [Thing(name='Mints', value=5, weight=25), Thing(name='Socks', value=10, weight=38), Thing(name='Tissues', value=15, weight=80), Thing(name='Phone', value=500, weight=200), Thing(name='Baseball Cap', value=100, weight=70), Thing(name='Laptop', value=500, weight=2200), Thing(name='Headphones', value=150, weight=160), Thing(name='Coffe Mug', value=60, weight=350), Thing(name='Notepad', value=40, weight=333), Thing(name='Water Bottle', value=30, weight=192)]
